In [ ]:
import os
import pandas as pd
import numpy as np
import folium
from datetime import datetime
import ast
import osmnx as ox
import networkx as nx
from tqdm import tqdm
from geopy.distance import geodesic

In [ ]:
G = ox.graph_from_place('Mannheim, Germany', network_type='drive')
location_csv = "altglascontainer-mannheim-clean.csv"
df_locations = pd.read_csv(location_csv)


In [ ]:
center_lat = df_locations['Latitude'].mean()
center_lon = df_locations['Longitude'].mean()

map_mannheim = folium.Map(location=[center_lat, center_lon], zoom_start=13)

In [ ]:
distance_matrix_df = pd.read_csv("distance_matrix.csv")
path_matrix_df = pd.read_csv("path_matrix.csv")

# Parse the path data (which is stored as string representations of lists)
for col in path_matrix_df.columns:
    path_matrix_df[col] = path_matrix_df[col].apply(lambda x: ast.literal_eval(x) if pd.notna(x) and x != '[]' else [])

distance_matrix = distance_matrix_df.values.tolist()
path_matrix = path_matrix_df.values.tolist()

print(f"Distance matrix shape: {len(distance_matrix)} x {len(distance_matrix[0])}")

In [ ]:
current_timestamp = datetime(2024, 1, 15, 00, 00)

def get_latest_fill_level(file_path, current_time):
    df = pd.read_csv(file_path)
    df['datetime'] = pd.to_datetime(df['Datum'] + ' ' + df['Uhrzeit'], format='%Y-%m-%d %H:%M')
    past_df = df[df['datetime'] <= current_time]
    if not past_df.empty:
        return past_df.sort_values('datetime').iloc[-1]['Füllstand (%)']
    return None

def sanitize_label(label):
    return label.replace(' ', '_').replace('#', '')

In [ ]:
ts_folder = "mannheim_container_ts"

container_states = []

for _, row in df_locations.iterrows():
    label = row['Label']
    safe_label = sanitize_label(label)
    ts_file = os.path.join(ts_folder, f"{safe_label}.csv")

    lat = row['Latitude']
    lon = row['Longitude']

    if not os.path.exists(ts_file):
        if label == "Depot #1":
            continue
        print(f"Missing timeseries file for: {label} (expected: {ts_file})")
        continue

    # Load entire timeseries CSV for this container
    df_ts = pd.read_csv(ts_file)
    
    # Filter for records until current timestamp
    df_ts["datetime"] = pd.to_datetime(df_ts["Datum"] + " " + df_ts["Uhrzeit"], format="%Y-%m-%d %H:%M")
    past_records = df_ts[df_ts["datetime"] <= current_timestamp]
    if past_records.empty:
        continue
    
    latest_row = past_records.sort_values("datetime").iloc[-1]

    fill_level = latest_row["Füllstand (%)"]
    container_size = latest_row["Containergröße (m³)"]  # Actual container size in m3
    container_type = latest_row["Container-Typ"]  # if needed, e.g., Weiß, Grün, Braun
    
    # Calculate actual volume filled in container at this timestamp
    fill_volume_m3 = (fill_level / 100.0) * container_size

    # Store extended state with actual load volume
    container_states.append({
        'Label': label,
        'Latitude': lat,
        'Longitude': lon,
        'Fill_Level': fill_level,
        'Container_Size_m3': container_size,
        'Fill_Volume_m3': fill_volume_m3,
        'Container_Type': container_type
    })



In [ ]:
print(f"Current timestamp: {current_timestamp}")
map_mannheim

In [ ]:
import random
from matplotlib import cm
class Truck:
    def __init__(self, truck_id, depot_lat, depot_lon, capacity_weiß, capacity_grün, capacity_braun, color=None):
        self.truck_id = truck_id
        self.color = color
        self.capacity_m3_weiß = capacity_weiß
        self.capacity_m3_grün = capacity_grün
        self.capacity_m3_braun = capacity_braun
        
        self.total_staff_cost_per_hour = 68.54  # € per hour
        self.cost_per_km = 1.80                 # € per km
        self.emission_per_km = 1.0              # kg CO₂ per km
        self.average_speed = 23.0               # km/h
        
        self.current_load_m3_weiß = 0.0
        self.current_load_m3_grün = 0.0
        self.current_load_m3_braun = 0.0
        
        self.depot_lat = depot_lat
        self.depot_lon = depot_lon
        
        self.location_timeseries = pd.DataFrame(columns=[
            'timestamp', 'latitude', 'longitude', 
            'load_m3_weiß', 'load_m3_grün', 'load_m3_braun'
        ])
    
    def empty_load(self):
        """Empties the truck loads for all glass types."""
        self.current_load_m3_weiß = 0.0
        self.current_load_m3_grün = 0.0
        self.current_load_m3_braun = 0.0
    
    def can_collect(self, volume_weiß, volume_grün, volume_braun):
        """Check if the truck can collect given volumes without exceeding capacity."""
        if (self.current_load_m3_weiß + volume_weiß > self.capacity_m3_weiß):
            return False
        if (self.current_load_m3_grün + volume_grün > self.capacity_m3_grün):
            return False
        if (self.current_load_m3_braun + volume_braun > self.capacity_m3_braun):
            return False
        return True
    
    def collect(self, volume_weiß, volume_grün, volume_braun):
        """Add volumes to the current load, assuming capacity check passed."""
        if not self.can_collect(volume_weiß, volume_grün, volume_braun):
            raise ValueError("Truck capacity exceeded!")
        self.current_load_m3_weiß += volume_weiß
        self.current_load_m3_grün += volume_grün
        self.current_load_m3_braun += volume_braun
    
    def add_position(self, timestamp, lat, lon, load_weiß=None, load_grün=None, load_braun=None):
        """Record a truck position and optionally update loads."""
        if load_weiß is not None:
            self.current_load_m3_weiß = load_weiß
        if load_grün is not None:
            self.current_load_m3_grün = load_grün
        if load_braun is not None:
            self.current_load_m3_braun = load_braun
        
        new_record = {
            'timestamp': timestamp,
            'latitude': lat,
            'longitude': lon,
            'load_m3_weiß': self.current_load_m3_weiß,
            'load_m3_grün': self.current_load_m3_grün,
            'load_m3_braun': self.current_load_m3_braun
        }
        
        self.location_timeseries = pd.concat(
            [self.location_timeseries, pd.DataFrame([new_record])],
            ignore_index=True
        )
    
    def get_current_location(self):
        """Return latest known truck location, or depot if none recorded."""
        if self.location_timeseries.empty:
            return (self.depot_lat, self.depot_lon)
        last_row = self.location_timeseries.iloc[-1]
        return (last_row.latitude, last_row.longitude)
    
    def get_location_timeseries(self):
        """Return the full recorded location timeseries DataFrame."""
        return self.location_timeseries

    def get_total_distance_km(self):
        """Calculate approximate total distance covered based on recorded positions."""
        if self.location_timeseries.shape[0] < 2:
            return 0.0
                
        total_distance = 0.0
        coords = list(zip(
            self.location_timeseries['latitude'],
            self.location_timeseries['longitude']
        ))
        
        for i in range(1, len(coords)):
            total_distance += geodesic(coords[i-1], coords[i]).kilometers
        
        return total_distance
    
    def estimate_total_cost(self):
        """
        Estimate total operation cost based on:
        - Staff hourly cost and total driving time (based on distance and speed)
        - Cost per km
        """
        distance_km = self.get_total_distance_km()
        total_hours = distance_km / self.average_speed if self.average_speed > 0 else 0
        
        staff_cost = total_hours * self.total_staff_cost_per_hour
        drive_cost = distance_km * self.cost_per_km
        
        return staff_cost + drive_cost
    
    def estimate_total_emissions(self):
        """Estimate total CO2 emissions based on distance covered."""
        distance_km = self.get_total_distance_km()
        return distance_km * self.emission_per_km
    
def get_n_folium_colors(n):
    folium_color_list = [
        "red", "blue", "green", "purple", "orange", "darkred",
        "lightred", "beige", "darkblue", "darkgreen", "cadetblue",
        "darkpurple", "white", "pink", "lightblue", "lightgreen",
        "gray", "black", "lightgray",
    ]
    if n <= len(folium_color_list):
        return folium_color_list[:n]
    else:
        # Repeat colors if trucks > available colors
        return [folium_color_list[i % len(folium_color_list)] for i in range(n)]

In [ ]:
num_nodes = len(df_locations)
priority = np.zeros(num_nodes)

for idx, row in df_locations.iterrows():
    label = row['Label']
    if label == "Depot #1":
        priority[idx] = 0
        continue

    container_state = next((item for item in container_states if item['Label'] == label), None)
    
    if container_state is not None and 'Fill_Level' in container_state:
        priority[idx] = container_state['Fill_Level']
    else:
        priority[idx] = 0
    
top_indices = np.argsort(priority)[::-1]

print("Top containers by priority (fill %) and index:")
for idx in top_indices[:50]:
    print(f"Index: {idx}, Label: {df_locations.iloc[idx]['Label']}, Fill: {priority[idx]:.2f}%")

In [ ]:
from datetime import timedelta

def plan_routes_with_rewards(trucks, container_states, df_locations, distance_matrix, start_time,
                             reward_per_m3=100):
    num_nodes = len(df_locations)
    depot_idx = 0

    label_to_idx = {label: idx for idx, label in enumerate(df_locations['Label'])}

    # Build demands per node as dict of glass types
    demands = [{} for _ in range(num_nodes)]
    for state in container_states:
        label = state['Label']
        idx = label_to_idx[label]
        typ = state['Container_Type'].lower()
        if typ not in demands[idx]:
            demands[idx][typ] = 0
        demands[idx][typ] += state.get('Fill_Volume_m3', 0)
    
    unvisited = set(i for i in range(1, num_nodes) if sum(demands[i].values()) > 0)
    routes = {truck.truck_id: [(depot_idx, start_time)] for truck in trucks}
    truck_loads = {truck.truck_id: {'weiß': 0.0, 'grün': 0.0, 'braun':0.0} for truck in trucks}
    truck_capacities = {
        truck.truck_id: {'weiß': truck.capacity_m3_weiß,
                         'grün': truck.capacity_m3_grün,
                         'braun': truck.capacity_m3_braun}
        for truck in trucks
    }
    cost_per_meter = trucks[0].cost_per_km / 1000  # €/m

    # While containers left to visit
    while unvisited:
        progress = False
        for truck in trucks:
            current_route = routes[truck.truck_id]
            current_node, current_time = current_route[-1]

            # Find feasible nodes this truck can pick up (obeying 3-type capacities separately)
            feasible_nodes = []
            for node in unvisited:
                node_demand = demands[node]
                feasible = True
                for typ in node_demand:
                    if truck_loads[truck.truck_id][typ] + node_demand[typ] > truck_capacities[truck.truck_id][typ]:
                        feasible = False
                        break
                if feasible:
                    feasible_nodes.append(node)

            if not feasible_nodes:
                # return to depot if not there
                if current_node != depot_idx:
                    dist_back = distance_matrix[current_node][depot_idx]
                    travel_time_back = timedelta(seconds=(dist_back / (truck.average_speed * 1000 / 3600)))
                    arrival_time = current_time + travel_time_back
                    routes[truck.truck_id].append((depot_idx, arrival_time))
                continue

            # For all feasible, pick one with maximal reward minus cost (profit)
            scores = []
            for node in feasible_nodes:
                node_demand = demands[node]
                reward = sum(node_demand[t] * reward_per_m3 for t in node_demand)
                dist = distance_matrix[current_node][node]
                travel_cost = dist * cost_per_meter
                net_gain = reward - travel_cost
                scores.append((net_gain, node))
            scores.sort(reverse=True)
            best_score, best_node = scores[0]  # greedy

            # “Travel” to this node and collect load
            dist_travel = distance_matrix[current_node][best_node]
            travel_time = timedelta(seconds=(dist_travel / (truck.average_speed * 1000 / 3600)))
            service_time = timedelta(minutes=5)

            arrival_time = current_time + travel_time
            departure_time = arrival_time + service_time

            routes[truck.truck_id].append((best_node, departure_time))

            # Update truck's loads by type
            node_demand = demands[best_node]
            for typ in node_demand:
                truck_loads[truck.truck_id][typ] += node_demand[typ]
            # Remove this location from all further trucks
            unvisited.remove(best_node)

            # Update time
            if len(routes[truck.truck_id]) > 0:
                routes[truck.truck_id][-1] = (best_node, departure_time)

            progress = True
        
        if not progress:
            break

    # After all done, each truck: if not at depot, return to depot
    for truck in trucks:
        current_route = routes[truck.truck_id]
        last_node, last_time = current_route[-1]
        if last_node != depot_idx:
            dist_back = distance_matrix[last_node][depot_idx]
            travel_time_back = timedelta(seconds=(dist_back / (truck.average_speed * 1000 / 3600)))
            arrival_time = last_time + travel_time_back
            current_route.append((depot_idx, arrival_time))
    return routes

In [ ]:
start_time = datetime(2024, 1, 15, 5, 0)

NUM_TRUCKS = 4
truck_colors = get_n_folium_colors(NUM_TRUCKS)
trucks = [
    Truck(f"Truck_{i+1}",
          depot_lat=df_locations.iloc[0]["Latitude"], 
          depot_lon=df_locations.iloc[0]["Longitude"],
          capacity_weiß=6.0,
          capacity_grün=6.0,
          capacity_braun=6.0,
          color=truck_colors[i]
    ) for i in range(NUM_TRUCKS)
]


routes_with_timestamps = plan_routes_with_rewards(trucks, container_states, df_locations, distance_matrix, start_time)

for truck_id, route in routes_with_timestamps.items():
    print(f"Route for {truck_id}:")
    for node_idx, timestamp in route:
        name = df_locations.iloc[node_idx]['Label']
        print(f"  Node {node_idx} | {name} at {timestamp}")

In [ ]:
route_map = folium.Map(location=[center_lat, center_lon], zoom_start=13)

# Use truck.colors to distinguish
for truck in trucks:
    truck_id = truck.truck_id
    this_color = truck.color
    route = routes_with_timestamps[truck_id]
    for node_idx, timestamp in route:
        loc = (df_locations.iloc[node_idx]['Latitude'], df_locations.iloc[node_idx]['Longitude'])
        label = df_locations.iloc[node_idx]['Label']
        folium.Marker(
            location=loc,
            popup=f"{label}<br>{timestamp}<br>{truck_id}",
            icon=folium.Icon(color='blue' if node_idx == 0 else truck.color, icon='truck' if node_idx == 0 else 'info-sign')
        ).add_to(route_map)
    # Draw lines between stops
    for idx in range(len(route)-1):
        start_idx = route[idx][0]
        end_idx = route[idx+1][0]
        path_node_ids = path_matrix[start_idx][end_idx]
        if not path_node_ids:
            continue
        path_coords = [(G.nodes[nid]['y'], G.nodes[nid]['x']) for nid in path_node_ids]
        folium.PolyLine(locations=path_coords, color=this_color, weight=4, opacity=0.8).add_to(route_map)

route_map